In [ ]:
# imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from scipy.stats import kendalltau
from scipy.stats import pearsonr
from sklearn.metrics import ndcg_score
import scipy.stats


In [ ]:
data = np.load('X-y.npz')
X = data['X']
y = data['y']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=17)

# SMOTE to balance my dataset
smote = SMOTE(random_state=42)
x_sm, y_sm = smote.fit_resample(X_train, y_train)

X_train = np.concatenate((X_train, x_sm), axis = 0)
y_train = np.concatenate((y_train, y_sm), axis = 0)

In [ ]:
svr = SVR(kernel='rbf', C=0.5, epsilon=0.2)
svr.fit(X_train, y_train)
y_pred_svr = svr.predict(X_val)

xgb_regressor = xgb.XGBRegressor()
xgb_regressor.fit(X_train, y_train)
y_pred_xgb = xgb_regressor.predict(X_val)

combined_prediction = (y_pred_svr + y_pred_xgb) / 2

print("\n")
print("actual: ", [ (num, '%.2f' % elem) for num, elem in enumerate(y_val) ])
print("predic: ", [ (num, '%.2f' % elem) for num, elem in enumerate(combined_prediction) ])
print("mean absolute error: ", mean_absolute_error(y_val, combined_prediction))
print("mean squared error: ", np.mean((y_val - combined_prediction)**2))

rounded_test = [round(y) for y in y_val]
rounded_predict = [round(y) for y in combined_prediction]
accuracyscore = accuracy_score(rounded_test, rounded_predict)
print("\n")
print("accuracy score: ", accuracyscore)
print("actual: ", rounded_test)
print("predic: ", rounded_predict)

In [ ]:
true_scores = np.array(y_val)
true_scores_sorted_indices = np.argsort(true_scores)

# now for xgboost regression + support vector regression
svr_scores = svr.predict(X_val)
xgboost_scores = xgb_regressor.predict(X_val)
combined_scores = (svr_scores + xgboost_scores) / 2
xgboost_svr_scores_sorted_indices = np.argsort(combined_scores)
tau_xg, p_value_xg = kendalltau(xgboost_svr_scores_sorted_indices, true_scores_sorted_indices)
print("\nxgboost and support vector regression classifier kendall tau: ", tau_xg, "xgboost and support vector regression classifier p_value: ", p_value_xg)
correlation, _ = pearsonr(combined_scores, true_scores)
ndcg = ndcg_score(np.array([combined_scores]), np.array([true_scores]))
ndcg_k10 = ndcg_score(np.array([combined_scores]), np.array([true_scores]), k = 10)
rev_ndcg_k10 = ndcg_score((1 / np.array([combined_scores])), (1 / np.array([true_scores])), k = 10)
print("xgboost and support vector regression classifier pearson correlation: ", correlation)
print("xgboost and support vector regression classifier ndcg: ", ndcg)
print("ndcg @ k = 10: ", ndcg_k10)
print("reverse ndcg @ k = 10: ", rev_ndcg_k10)